In [1]:
import numpy as np
import pandas as pd
from tqdm.notebook import trange, tqdm
import multiprocessing as mp
import itertools
import copy
from itertools import combinations

In [2]:
growth_df = pd.read_csv('C:\\Users\marku\Documents\Pure Piggy Growth - Sheet1.csv')
buff_df = pd.read_csv('C:\\Users\marku\Documents\Piggy_Buffs - Sheet1.csv')

In [3]:
class Piggy:
    def __init__(self, breed, level):
        assert 0.98 < sum(breed.values()) < 1.02        
        self.breed = breed
        self.level = level
            
    def starting_stats(self):
        starting_stats = {
            "AP": self.breed.get("Warrior")*20+self.breed.get("Gunslinger")*38+self.breed.get("Magician")*50
            +self.breed.get("Manga")*35+self.breed.get("Musician")*25+self.breed.get("Strategist")*25,
            "AS": self.breed.get("Warrior")*1+self.breed.get("Gunslinger")*1+self.breed.get("Magician")*0.85
            +self.breed.get("Manga")*1+self.breed.get("Musician")*1+self.breed.get("Strategist")*1,
            "DF": self.breed.get("Warrior")*90+self.breed.get("Gunslinger")*30+self.breed.get("Magician")*15
            +self.breed.get("Manga")*50+self.breed.get("Musician")*40+self.breed.get("Strategist")*40,
            "Regen": self.breed.get("Warrior")*7+self.breed.get("Gunslinger")*6+self.breed.get("Magician")*6
            +self.breed.get("Manga")*6+self.breed.get("Musician")*15+self.breed.get("Strategist")*8,
            "HP": self.breed.get("Warrior")*550+self.breed.get("Gunslinger")*550+self.breed.get("Magician")*550
            +self.breed.get("Manga")*500+self.breed.get("Musician")*550+self.breed.get("Strategist")*650,
            "Crit": self.breed.get("Warrior")*0.08+self.breed.get("Gunslinger")*0.08+self.breed.get("Magician")*0.04
            +self.breed.get("Manga")*0.25+self.breed.get("Musician")*0.04+self.breed.get("Strategist")*0.07
        }
        return starting_stats

    
    def estimate_stats(self):

        df = growth_df.copy()
        conditions = [
            (df['Clan'] == "Warrior"),
            (df['Clan'] == "Gunslinger"),
            (df['Clan'] == "Magician"),
            (df['Clan'] == "Manga"),
            (df['Clan'] == "Musician"),
            (df['Clan'] == "Strategist"),
            ]

        # create a list of the values we want to assign for each condition
        values = [self.breed.get("Warrior"), self.breed.get("Gunslinger"), self.breed.get("Magician"),
                  self.breed.get("Manga"), self.breed.get("Musician"), self.breed.get("Strategist")]

        # create a new column and use np.select to assign values to it using our lists as arguments
        df['Breed'] = np.select(conditions, values)    
        
        df["Amount_per_breed"] = df["Amount"] * df["Breed"]
        df["Jump_per_breed"] = df["Jump"] * df["Breed"]
        
        df1 = df[df['Start Level'] == 1] #Just stats of level1 line
        df10 = df[df['Start Level'] == 10] #Just stats of level10 line

        amount_lvl1 = df1.groupby('Stat')['Amount_per_breed'].sum() #amount lvl1
        jump_lvl1 = df1.groupby('Stat')['Jump_per_breed'].sum() #jumps lvl1
        
        amount_lvl10 = df10.groupby('Stat')['Amount_per_breed'].sum() #amount lvl10
        jump_lvl10 = df10.groupby('Stat')['Jump_per_breed'].sum() #jumps lvl10
        
        stats = self.starting_stats()
        
        for stat in self.starting_stats():
            current_jump = 0
            for i in range(self.level-1):
                last_jump = current_jump
                current_jump += jump_lvl1[stat]
                if np.floor(current_jump) - np.floor(last_jump) > 0:
                    stats[stat] += amount_lvl1[stat]
            if self.level > 10:
                current_jump = 0
                for i in range(self.level-10):
                    last_jump = current_jump
                    current_jump += jump_lvl10[stat]
                    if np.floor(current_jump) - np.floor(last_jump) > 0:
                        stats[stat] = stats[stat] + amount_lvl10[stat]

        return stats
    
    def display_rounded_stats(self):
        return {key : round(self.estimate_stats()[key], 4) for key in self.estimate_stats()}

In [4]:
df_booster_buffs = buff_df.loc[buff_df['Clan'] == "Booster"]
df_booster_buff1 = df_booster_buffs.loc[df_booster_buffs["Buff"] == "BB2"] #HP
df_booster_buff2 = df_booster_buffs.loc[df_booster_buffs["Buff"] == "BB5"] #ATK

df_booster_buff1 = df_booster_buff1.drop('Name', axis=1)
df_booster_buff1 = df_booster_buff1.drop('Clan', axis=1)
df_booster_buff1 = df_booster_buff1.drop("Required Level", axis = 1)

df_booster_buff2 = df_booster_buff2.drop('Name', axis=1)
df_booster_buff2 = df_booster_buff2.drop('Clan', axis=1)
df_booster_buff2 = df_booster_buff2.drop('Required Level', axis=1)
boosters = pd.concat([df_booster_buff1, df_booster_buff2])
boosters

,Buff,AP,AS,DF,Regen,HP,Crit
177,BB2,0.00,0.0,0.0,0.0,0.06,0.0
178,BB5,0.06,0.0,0.0,0.0,0.00,0.0


In [5]:
### reasonable Warrior buffs (really doubt we can use more than one)
warrior_buffs = buff_df.loc[buff_df['Clan'] == "Warrior"]
possible_buffs = warrior_buffs.loc[warrior_buffs['Required Level'] <= 8]
reasonable_buffs_warrior = possible_buffs.loc[warrior_buffs['AP'] >= 0.03]
reasonable_buffs_warrior

,Buff,Name,Clan,Required Level,AP,AS,DF,Regen,HP,Crit
6,C7,Bow and Arrow,Warrior,1,0.04,0.0,0.04,0.0,0.0,0.0
37,B2,Axe,Warrior,5,0.03,0.0,0.03,0.0,0.0,0.0


In [6]:
### reasonable Gun buffs: let's allow two
gun_buffs = buff_df.loc[buff_df['Clan'] == "Gunslinger"]
possible_buffs = gun_buffs.loc[gun_buffs['Required Level'] <=  8]
reasonable_buffs_gun = possible_buffs.iloc[[0,1]]
reasonable_buffs_gun

,Buff,Name,Clan,Required Level,AP,AS,DF,Regen,HP,Crit
1,C2,Uzi,Gunslinger,1,0.02,0.02,0.0,0.0,0.0,0.0
7,C8,Grenade,Gunslinger,1,0.04,0.04,0.0,0.0,0.0,0.0


In [7]:
### reasonable mage buffs, mage isn't really reasonable, we'll exclude it for now

In [8]:
### reasonable manga buffs: allow B11 and C10
manga_buffs = buff_df.loc[buff_df['Clan'] == "Manga"]
possible_buffs = manga_buffs.loc[manga_buffs['Required Level'] <=  8]
reasonable_buffs_manga = possible_buffs.iloc[[1,3]]
reasonable_buffs_manga

,Buff,Name,Clan,Required Level,AP,AS,DF,Regen,HP,Crit
9,C10,Punch Glove,Manga,1,0.04,0.0,0.0,0.0,0.0,0.04
82,B11,Pin,Manga,5,0.03,0.0,0.0,0.0,0.0,0.03


In [9]:
### reasonable strat buffs: C11, B14, C5
strat_buffs = buff_df.loc[buff_df['Clan'] == "Strategist"]
possible_buffs = strat_buffs.loc[strat_buffs['Required Level'] <=  8]
reasonable_buffs_strat = possible_buffs.iloc[[1,3]]
reasonable_buffs_strat

,Buff,Name,Clan,Required Level,AP,AS,DF,Regen,HP,Crit
10,C11,AI Glasses,Strategist,1,0.04,0.0,0.0,0.0,0.04,0.0
97,B14,Magnifying Glass,Strategist,5,0.03,0.0,0.0,0.0,0.03,0.0


In [10]:
### No Musician considered

In [11]:
### Possible Armour
armour_buffs = buff_df.loc[buff_df['Clan'] == 'Armour']
possible_buffs = armour_buffs.loc[armour_buffs['Required Level'] <=  8]
somewhat_reasonable_buffs = possible_buffs.loc[armour_buffs['Required Level'] >=  6]
reasonable_buffs_armour = somewhat_reasonable_buffs.iloc[[1,2,3,4,6,7]]
reasonable_buffs_armour

,Buff,Name,Clan,Required Level,AP,AS,DF,Regen,HP,Crit
144,BC5,Phalanx Boots,Armour,6,0.00,0.00,0.00,0.00,0.03,0.03
145,BC12,Traveller Robe,Armour,6,0.03,0.00,0.00,0.03,0.00,0.00
146,BC7,Bandit Mask,Armour,7,0.03,0.02,0.00,0.00,0.00,0.00
147,BC11,Guerrilla Armor,Armour,7,0.00,0.00,0.03,0.00,0.03,0.00
149,BC4,Mystic Robe,Armour,8,0.00,0.03,0.00,0.00,0.00,0.05
150,BC6,Iron Armor,Armour,8,0.00,0.00,0.05,0.05,0.00,0.00


In [12]:
def Combos(df_Buffs):
    Combos = []
    for index in list(combinations(df_Buffs.index,4)):
        Combos.append(df_Buffs.loc[index,:])
    for i in range(len(Combos)):
        Combos[i] = Combos[i].drop('Required Level', axis=1)
        Combos[i] = Combos[i].drop('Name', axis=1)
        Combos[i] = Combos[i].drop('Clan', axis=1)
        Combos[i]["AP"] += 1
        Combos[i]["AS"] += 1
        Combos[i]["DF"] += 1
        Combos[i]["Regen"] += 1
        Combos[i]["HP"] += 1
        Combos[i]["Crit"] += 1
    for i in range(len(Combos)):
        newrow = pd.DataFrame({
            'Buff': Combos[i].iloc[0][0] + ' ' + Combos[i].iloc[1][0] + ' ' + Combos[i].iloc[2][0] + ' ' + Combos[i].iloc[3][0],
            'AP': Combos[i]["AP"].product(),
            'AS': Combos[i]["AS"].product(),
            'DF': Combos[i]["DF"].product(),
            'Regen': Combos[i]["Regen"].product(),
            'HP': Combos[i]["HP"].product(),
            'Crit': Combos[i]["Crit"].product()
        } ,index=[0])

        #Combos[i] = pd.concat([newrow,Combos[i].loc[:]]).reset_index()
        Combos[i] = newrow

    ## Add boosters
    Combos_BB2 = copy.deepcopy(Combos)
    Combos_BB5 = copy.deepcopy(Combos)

    for i in range(len(Combos)):
        Combos_BB2[i]["HP"] *= 1.06 #HP
        Combos_BB2[i]["Buff"] += " " + "BB2"

        Combos_BB5[i]["AP"] *= 1.06 #AP
        Combos_BB5[i]["Buff"] += " " + "BB5"

    Combos = [*Combos_BB2, *Combos_BB5]
    df = pd.concat(Combos).reset_index()
    df = df.drop('index', axis=1)
    return df

In [13]:
df_Gun_Strat_Combos = Combos(pd.concat([reasonable_buffs_armour, reasonable_buffs_gun, reasonable_buffs_strat]))
df_Gun_Warrior_Combos = Combos(pd.concat([reasonable_buffs_armour, reasonable_buffs_gun, reasonable_buffs_warrior]))
df_Strat_Warrior_Combos = Combos(pd.concat([reasonable_buffs_armour, reasonable_buffs_strat, reasonable_buffs_warrior]))
df_Gun_Strat_Manga_Combos = Combos(pd.concat([reasonable_buffs_armour, reasonable_buffs_strat, reasonable_buffs_gun, reasonable_buffs_manga]))
df_Gun_Strat_Warrior_Combos = Combos(pd.concat([reasonable_buffs_armour, reasonable_buffs_strat, reasonable_buffs_gun, reasonable_buffs_warrior]))
df_Gun_Strat_Warrior_Manga_Combos = Combos(pd.concat([reasonable_buffs_armour, reasonable_buffs_gun, reasonable_buffs_strat, reasonable_buffs_manga, reasonable_buffs_warrior]))                                                       

In [14]:
def create_piggies(warrior=0, gun=0, magician=0, manga=0, musician=0, strat=0, level=15):
    assert 0.999 < sum([warrior, gun, magician, manga, musician, strat]) < 1.001
    current_breed = {
        "Warrior": warrior,
        "Gunslinger": gun,
        "Magician": magician,
        "Manga": manga,
        "Musician": musician,
        "Strategist": strat
    }

    return Piggy(breed=current_breed, level=level)

def Merge(dict_1, dict_2):
    result = dict_1 | dict_2
    return result

In [15]:
def create_piggy_df(dist = 1, level = 8, gunMax = 100, gunMin = 0, stratMax = 100, stratMin = 0, warriorMax = 100, warriorMin = 0, mangaMax = 100, mangaMin = 0):
    piggy_array = []
    for gun in range(gunMin,min([gunMax+dist, dist+100-warriorMin-stratMin-mangaMin]),dist):
        for warrior in range(warriorMin,min([warriorMax+dist, dist+100-gun-stratMin-mangaMin]),dist):
            for manga in range(max([mangaMin, 100-gun-warrior-stratMax]),min([mangaMax+dist, dist+100-gun-warrior-stratMin]),dist):
                piggy_array.append(create_piggies(gun = gun/100, warrior = warrior/100, manga = manga/100,
                                                  strat = 1-(gun+warrior+manga)/100, level = level))
    piggy_list = []
    for pig in piggy_array:
        piggy_list.append(Merge(pig.estimate_stats(),pig.breed))
    return pd.DataFrame(piggy_list)

In [16]:
def calculate_buffs(piggy_df, buff_df):
    df = piggy_df.merge(buff_df, how='cross')
    df["average Battle Power"] = ((df["HP_x"]*df["HP_y"] + 
                                 3*df["DF_x"]*df["DF_y"] +
                                 10*df["Regen_x"]*df["Regen_y"]) +
                                 (df["AP_x"]*df["AP_y"] *
                                 df["AS_x"]*df["AS_y"]) *
                                 (10 + 5 * df["Crit_x"]*df["Crit_y"])
                                 )
    return df.sort_values("average Battle Power", ascending=False)

In [17]:
piggies_strat_gun = create_piggy_df(dist = 1, level = 8, gunMax = 80, gunMin = 20, stratMax = 80, stratMin = 20, warriorMax = 19, mangaMax = 19)
results_strat_gun = calculate_buffs(piggies_strat_gun, df_Gun_Strat_Combos).head(10)
results_strat_gun

,AP_x,AS_x,DF_x,Regen_x,HP_x,Crit_x,Warrior,Gunslinger,Magician,Manga,Musician,Strategist,Buff,AP_y,AS_y,DF_y,Regen_y,HP_y,Crit_y,average Battle Power
5590794,59.596,1.2074,63.820,9.720,931.25,0.11710,0.19,0.53,0,0.06,0,0.22,BC7 C8 C11 B14 BB2,1.147469,1.0608,1.0,1.0,1.135472,1.0,2273.227471
5588274,58.996,1.2122,66.180,8.750,942.35,0.10720,0.19,0.53,0,0.00,0,0.28,BC7 C8 C11 B14 BB2,1.147469,1.0608,1.0,1.0,1.135472,1.0,2273.216563
5591214,59.696,1.2066,63.950,9.715,929.40,0.11875,0.19,0.53,0,0.07,0,0.21,BC7 C8 C11 B14 BB2,1.147469,1.0608,1.0,1.0,1.135472,1.0,2273.130571
5591634,59.796,1.2058,64.080,9.710,927.55,0.12040,0.19,0.53,0,0.08,0,0.20,BC7 C8 C11 B14 BB2,1.147469,1.0608,1.0,1.0,1.135472,1.0,2273.033072
5584074,58.976,1.2126,65.715,8.760,943.35,0.10710,0.18,0.53,0,0.00,0,0.29,BC7 C8 C11 B14 BB2,1.147469,1.0608,1.0,1.0,1.135472,1.0,2273.005128
5586594,59.576,1.2078,63.350,9.730,932.25,0.11700,0.18,0.53,0,0.06,0,0.23,BC7 C8 C11 B14 BB2,1.147469,1.0608,1.0,1.0,1.135472,1.0,2273.005057
5587014,59.676,1.2070,63.480,9.725,930.40,0.11865,0.18,0.53,0,0.07,0,0.22,BC7 C8 C11 B14 BB2,1.147469,1.0608,1.0,1.0,1.135472,1.0,2272.908832
5587434,59.776,1.2062,63.610,9.720,928.55,0.12030,0.18,0.53,0,0.08,0,0.21,BC7 C8 C11 B14 BB2,1.147469,1.0608,1.0,1.0,1.135472,1.0,2272.812008
5579454,58.956,1.2130,65.250,8.770,944.35,0.10700,0.17,0.53,0,0.00,0,0.30,BC7 C8 C11 B14 BB2,1.147469,1.0608,1.0,1.0,1.135472,1.0,2272.793488
5582394,59.656,1.2074,63.010,9.735,931.40,0.11855,0.17,0.53,0,0.07,0,0.23,BC7 C8 C11 B14 BB2,1.147469,1.0608,1.0,1.0,1.135472,1.0,2272.686886


In [18]:
piggies_warrior_gun = create_piggy_df(dist = 1, level = 8, gunMax = 80, gunMin = 20, stratMax = 19, warriorMin = 20, warriorMax = 80, mangaMax = 19)
results_warrior_gun = calculate_buffs(piggies_warrior_gun, df_Gun_Warrior_Combos).head(10)
results_warrior_gun

,AP_x,AS_x,DF_x,Regen_x,HP_x,Crit_x,Warrior,Gunslinger,Magician,Manga,Musician,Strategist,Buff,AP_y,AS_y,DF_y,Regen_y,HP_y,Crit_y,average Battle Power
3023732,57.124,1.18675,88.800,9.63,931.15,0.1100,0.63,0.37,0,0.0,0,0.00,BC7 BC11 C8 C7 BB2,1.114048,1.0608,1.0712,1.0,1.0918,1.0,2243.513280
3022892,57.104,1.18705,88.335,9.64,932.15,0.1099,0.62,0.37,0,0.0,0,0.01,BC7 BC11 C8 C7 BB2,1.114048,1.0608,1.0712,1.0,1.0918,1.0,2243.088367
3021632,57.084,1.18735,87.870,9.65,933.15,0.1098,0.61,0.37,0,0.0,0,0.02,BC7 BC11 C8 C7 BB2,1.114048,1.0608,1.0712,1.0,1.0918,1.0,2242.663313
3019952,57.064,1.18765,87.405,9.66,934.15,0.1097,0.60,0.37,0,0.0,0,0.03,BC7 BC11 C8 C7 BB2,1.114048,1.0608,1.0712,1.0,1.0918,1.0,2242.238116
3017852,57.044,1.18795,86.940,9.67,935.15,0.1096,0.59,0.37,0,0.0,0,0.04,BC7 BC11 C8 C7 BB2,1.114048,1.0608,1.0712,1.0,1.0918,1.0,2241.812778
3015332,57.024,1.18825,86.475,9.68,936.15,0.1095,0.58,0.37,0,0.0,0,0.05,BC7 BC11 C8 C7 BB2,1.114048,1.0608,1.0712,1.0,1.0918,1.0,2241.387299
3191732,57.276,1.18450,88.200,9.62,930.10,0.1100,0.62,0.38,0,0.0,0,0.00,BC7 BC11 C8 C7 BB2,1.114048,1.0608,1.0712,1.0,1.0918,1.0,2240.981008
3012392,57.004,1.18855,86.010,9.69,937.15,0.1094,0.57,0.37,0,0.0,0,0.06,BC7 BC11 C8 C7 BB2,1.114048,1.0608,1.0712,1.0,1.0918,1.0,2240.961677
3190892,57.256,1.18480,87.735,9.63,931.10,0.1099,0.61,0.38,0,0.0,0,0.01,BC7 BC11 C8 C7 BB2,1.114048,1.0608,1.0712,1.0,1.0918,1.0,2240.557195
3009032,56.984,1.18885,85.545,9.70,938.15,0.1093,0.56,0.37,0,0.0,0,0.07,BC7 BC11 C8 C7 BB2,1.114048,1.0608,1.0712,1.0,1.0918,1.0,2240.535914


In [19]:
piggies_strat_warrior = create_piggy_df(dist = 1, level = 8, gunMax = 19, stratMin = 20, stratMax = 80, warriorMin = 20, warriorMax = 80, mangaMax = 19)
results_strat_warrior = calculate_buffs(piggies_strat_warrior, df_Strat_Warrior_Combos).head(10)
results_strat_warrior

,AP_x,AS_x,DF_x,Regen_x,HP_x,Crit_x,Warrior,Gunslinger,Magician,Manga,Musician,Strategist,Buff,AP_y,AS_y,DF_y,Regen_y,HP_y,Crit_y,average Battle Power
2469371,51.860,1.08825,98.700,9.150,984.75,0.10800,0.75,0.05,0,0.00,0,0.20,BC11 C11 B14 C7 BB2,1.114048,1.0,1.0712,1.0,1.169536,1.00,2223.065859
2468531,51.840,1.08835,98.235,9.160,985.75,0.10790,0.74,0.05,0,0.00,0,0.21,BC11 C11 B14 C7 BB2,1.114048,1.0,1.0712,1.0,1.169536,1.00,2222.614949
2467271,51.820,1.08845,97.770,9.170,986.75,0.10780,0.73,0.05,0,0.00,0,0.22,BC11 C11 B14 C7 BB2,1.114048,1.0,1.0712,1.0,1.169536,1.00,2222.164010
2468111,52.020,1.08805,94.920,10.160,983.05,0.11110,0.73,0.05,0,0.02,0,0.20,BC11 C11 B14 C7 BB2,1.114048,1.0,1.0712,1.0,1.169536,1.00,2221.929965
2465591,51.800,1.08855,97.305,9.180,987.75,0.10770,0.72,0.05,0,0.00,0,0.23,BC11 C11 B14 C7 BB2,1.114048,1.0,1.0712,1.0,1.169536,1.00,2221.713043
2466851,52.100,1.08795,94.580,10.165,982.20,0.11265,0.72,0.05,0,0.03,0,0.20,BC11 C11 B14 C7 BB2,1.114048,1.0,1.0712,1.0,1.169536,1.00,2221.344935
2851571,52.012,1.08750,98.100,9.140,983.70,0.10800,0.74,0.06,0,0.00,0,0.20,BC11 C11 B14 C7 BB2,1.114048,1.0,1.0712,1.0,1.169536,1.00,2221.293942
2463491,51.780,1.08865,96.840,9.190,988.75,0.10760,0.71,0.05,0,0.00,0,0.24,BC11 C11 B14 C7 BB2,1.114048,1.0,1.0712,1.0,1.169536,1.00,2221.262047
2469238,51.860,1.08825,98.700,9.150,984.75,0.10800,0.75,0.05,0,0.00,0,0.20,BC5 BC11 C11 B14 BB2,1.071200,1.0,1.0300,1.0,1.204622,1.03,2220.909131
2464751,52.080,1.08805,94.110,10.175,983.20,0.11255,0.71,0.05,0,0.03,0,0.21,BC11 C11 B14 C7 BB2,1.114048,1.0,1.0712,1.0,1.169536,1.00,2220.877743


In [20]:
piggies_gun_strat_manga = create_piggy_df(dist = 1, level = 8, gunMax = 60, gunMin = 20, stratMax = 60, stratMin = 20, warriorMax = 19, mangaMax = 60, mangaMin = 20)
results_gun_strat_manga = calculate_buffs(piggies_gun_strat_manga, df_Gun_Strat_Manga_Combos).head(10)
results_gun_strat_manga

,AP_x,AS_x,DF_x,Regen_x,HP_x,Crit_x,Warrior,Gunslinger,Magician,Manga,Musician,Strategist,Buff,AP_y,AS_y,DF_y,Regen_y,HP_y,Crit_y,average Battle Power
10380500,60.756,1.20100,60.00,9.770,917.35,0.13900,0.07,0.53,0,0.20,0,0.20,BC7 C11 B14 C8 BB2,1.147469,1.0608,1.0,1.0,1.135472,1.0,2269.246502
10378520,60.736,1.20140,59.53,9.780,918.35,0.13890,0.06,0.53,0,0.20,0,0.21,BC7 C11 B14 C8 BB2,1.147469,1.0608,1.0,1.0,1.135472,1.0,2269.031136
10379510,60.836,1.20060,59.66,9.775,916.50,0.14055,0.06,0.53,0,0.21,0,0.20,BC7 C11 B14 C8 BB2,1.147469,1.0608,1.0,1.0,1.135472,1.0,2268.934384
10375550,60.716,1.20180,59.06,9.790,919.35,0.13880,0.05,0.53,0,0.20,0,0.22,BC7 C11 B14 C8 BB2,1.147469,1.0608,1.0,1.0,1.135472,1.0,2268.815562
8196560,58.644,1.18555,68.24,9.950,930.75,0.14520,0.19,0.37,0,0.24,0,0.20,BC7 C11 B14 C8 BB2,1.147469,1.0608,1.0,1.0,1.135472,1.0,2268.788898
8195570,58.544,1.18615,68.11,9.955,932.60,0.14355,0.19,0.37,0,0.23,0,0.21,BC7 C11 B14 C8 BB2,1.147469,1.0608,1.0,1.0,1.135472,1.0,2268.762922
8194580,58.444,1.18675,67.98,9.960,934.45,0.14190,0.19,0.37,0,0.22,0,0.22,BC7 C11 B14 C8 BB2,1.147469,1.0608,1.0,1.0,1.135472,1.0,2268.737057
10376540,60.816,1.20100,59.19,9.785,917.50,0.14045,0.05,0.53,0,0.21,0,0.21,BC7 C11 B14 C8 BB2,1.147469,1.0608,1.0,1.0,1.135472,1.0,2268.719499
8193590,58.344,1.18735,67.85,9.965,936.30,0.14025,0.19,0.37,0,0.21,0,0.23,BC7 C11 B14 C8 BB2,1.147469,1.0608,1.0,1.0,1.135472,1.0,2268.711307
8192600,58.244,1.18795,67.72,9.970,938.15,0.13860,0.19,0.37,0,0.20,0,0.24,BC7 C11 B14 C8 BB2,1.147469,1.0608,1.0,1.0,1.135472,1.0,2268.685676


In [21]:
piggies_strat_gun_warrior = create_piggy_df(dist = 1, level = 8, gunMax = 60, gunMin = 20, stratMax = 60, stratMin = 20, warriorMin = 20, warriorMax = 60, mangaMax = 19)
results_strat_gun_warrior = calculate_buffs(piggies_strat_gun_warrior, df_Gun_Strat_Warrior_Combos).head()
results_strat_gun_warrior

,AP_x,AS_x,DF_x,Regen_x,HP_x,Crit_x,Warrior,Gunslinger,Magician,Manga,Musician,Strategist,Buff,AP_y,AS_y,DF_y,Regen_y,HP_y,Crit_y,average Battle Power
8196693,56.724,1.19275,79.500,8.83,951.15,0.1080,0.43,0.37,0,0.00,0,0.20,C11 B14 C8 C7 BB2,1.158610,1.0400,1.04,1.0,1.135472,1.0,2275.609874
8194713,56.704,1.19305,79.035,8.84,952.15,0.1079,0.42,0.37,0,0.00,0,0.21,C11 B14 C8 C7 BB2,1.158610,1.0400,1.04,1.0,1.135472,1.0,2275.266870
10377530,59.316,1.20820,66.120,9.68,932.65,0.1111,0.25,0.53,0,0.02,0,0.20,BC7 C11 B14 C8 BB2,1.147469,1.0608,1.00,1.0,1.135472,1.0,2274.955473
8191743,56.684,1.19335,78.570,8.85,953.15,0.1078,0.41,0.37,0,0.00,0,0.22,C11 B14 C8 C7 BB2,1.158610,1.0400,1.04,1.0,1.135472,1.0,2274.923722
10380500,59.156,1.20900,69.900,8.67,934.35,0.1080,0.27,0.53,0,0.00,0,0.20,BC7 C11 B14 C8 BB2,1.147469,1.0608,1.00,1.0,1.135472,1.0,2274.900683


In [23]:
piggies_strat_manga_warrior = create_piggy_df(dist = 1, level = 8, mangaMax = 60, mangaMin = 20, stratMax = 60, stratMin = 20, warriorMin = 20, warriorMax = 60, gunMax = 19)
results_strat_manga_warrior = calculate_buffs(piggies_strat_manga_warrior, df_Gun_Strat_Warrior_Combos).head()
results_strat_manga_warrior

,AP_x,AS_x,DF_x,Regen_x,HP_x,Crit_x,Warrior,Gunslinger,Magician,Manga,Musician,Strategist,Buff,AP_y,AS_y,DF_y,Regen_y,HP_y,Crit_y,average Battle Power
4524706,53.46,1.08625,88.80,10.250,967.75,0.13900,0.55,0.05,0,0.20,0,0.20,BC11 C11 B14 C8 BB2,1.114048,1.04,1.03,1.0,1.169536,1.0,2228.286794
4522726,53.44,1.08635,88.33,10.260,968.75,0.13890,0.54,0.05,0,0.20,0,0.21,BC11 C11 B14 C8 BB2,1.114048,1.04,1.03,1.0,1.169536,1.0,2227.867416
4523716,53.54,1.08615,88.46,10.255,966.90,0.14055,0.54,0.05,0,0.21,0,0.20,BC11 C11 B14 C8 BB2,1.114048,1.04,1.03,1.0,1.169536,1.0,2227.824716
4519756,53.42,1.08645,87.86,10.270,969.75,0.13880,0.53,0.05,0,0.20,0,0.22,BC11 C11 B14 C8 BB2,1.114048,1.04,1.03,1.0,1.169536,1.0,2227.448008
4520746,53.52,1.08625,87.99,10.265,967.90,0.14045,0.53,0.05,0,0.21,0,0.21,BC11 C11 B14 C8 BB2,1.114048,1.04,1.03,1.0,1.169536,1.0,2227.405268


In [24]:
piggies_strat_gun_warrior_manga = create_piggy_df(dist = 1, level = 8, gunMax = 40, gunMin = 20, stratMax = 40, stratMin = 20, warriorMax = 40, warriorMin = 20, mangaMax = 40, mangaMin = 20)
results_strat_gun_warrior_manga = calculate_buffs(piggies_strat_gun_warrior_manga, df_Gun_Strat_Warrior_Manga_Combos).head(10)
results_strat_gun_warrior_manga

,AP_x,AS_x,DF_x,Regen_x,HP_x,Crit_x,Warrior,Gunslinger,Magician,Manga,Musician,Strategist,Buff,AP_y,AS_y,DF_y,Regen_y,HP_y,Crit_y,average Battle Power
3524156,58.324,1.18675,69.60,9.930,934.15,0.13900,0.23,0.37,0,0.20,0,0.20,BC7 C8 C11 B14 BB2,1.147469,1.0608,1.00,1.0,1.135472,1.0,2269.878304
3520152,58.304,1.18705,69.13,9.940,935.15,0.13890,0.22,0.37,0,0.20,0,0.21,BC7 C8 C11 B14 BB2,1.147469,1.0608,1.00,1.0,1.135472,1.0,2269.580370
3514146,58.284,1.18735,68.66,9.950,936.15,0.13880,0.21,0.37,0,0.20,0,0.22,BC7 C8 C11 B14 BB2,1.147469,1.0608,1.00,1.0,1.135472,1.0,2269.282287
3524488,58.324,1.18675,69.60,9.930,934.15,0.13900,0.23,0.37,0,0.20,0,0.20,C8 C11 B14 C7 BB2,1.158610,1.0400,1.04,1.0,1.135472,1.0,2269.138911
3512144,58.564,1.18585,68.58,9.945,931.60,0.14365,0.20,0.37,0,0.23,0,0.20,BC7 C8 C11 B14 BB2,1.147469,1.0608,1.00,1.0,1.135472,1.0,2269.060000
3510142,58.464,1.18645,68.45,9.950,933.45,0.14200,0.20,0.37,0,0.22,0,0.21,BC7 C8 C11 B14 BB2,1.147469,1.0608,1.00,1.0,1.135472,1.0,2269.034570
3508140,58.364,1.18705,68.32,9.955,935.30,0.14035,0.20,0.37,0,0.21,0,0.22,BC7 C8 C11 B14 BB2,1.147469,1.0608,1.00,1.0,1.135472,1.0,2269.009254
3506138,58.264,1.18765,68.19,9.960,937.15,0.13870,0.20,0.37,0,0.20,0,0.23,BC7 C8 C11 B14 BB2,1.147469,1.0608,1.00,1.0,1.135472,1.0,2268.984056
3520484,58.304,1.18705,69.13,9.940,935.15,0.13890,0.22,0.37,0,0.20,0,0.21,C8 C11 B14 C7 BB2,1.158610,1.0400,1.04,1.0,1.135472,1.0,2268.785822
3514478,58.284,1.18735,68.66,9.950,936.15,0.13880,0.21,0.37,0,0.20,0,0.22,C8 C11 B14 C7 BB2,1.158610,1.0400,1.04,1.0,1.135472,1.0,2268.432586
